In [2]:
!pip install transformers torch accelerate SentencePiece huggingface_hub

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/c1/bd/f64d67df4d3b05a460f281defe830ffab6d7940b7ca98ec085e94e024781/transformers-4.34.1-py3-none-any.whl.metadata
  Using cached transformers-4.34.1-py3-none-any.whl.metadata (121 kB)
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/6d/13/b5e8bacd980b2195f8a1741ce11cbb9146568607795d5e4ff510dcff1064/torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.16.4 from https://files.pythonhosted.org/packages/ef/b5/b6107bd65fa4c96fdf00e4733e2fe5729bb9e5e09997f63074bb43d3ab28/huggingface_hub-0.18.0-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.18.0-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for tokenizers<0.15,>=0.14 from https://files.pythonhosted.org/packages/a7/7b/c1f643eb086b6c5c

In [1]:
## 加载模型
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaForCausalLM
import torch

device = torch.device("cuda")


model_path="/home/ec2-user/SageMaker/model_weights/Ziya2-13b-chat"

model = LlamaForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [26]:
#给定system角色。
instruction = """
仅仅只把下面片段中涉及到色情、政治、暴力、恐怖、隐私、辱骂、违法等相关含义的文字替换为==，其他的文字需要保持不变。下面给出一个例子，请参考给出的例子作答。 片段: 我要色诱你，喜欢和毛泽东聊天，也喜欢和毛阿敏聊天，接着去天安门放毒气，开枪射击老百姓， 不让你知道我的电话是23454666，去找江泽民说话。 替换为: 我要==你，喜欢和==聊天，也喜欢和毛阿敏聊天，接着去天安门==，==老百姓，不让你知道我的电话是==，去找==说话。
"""
question = """
片段：
在游戏世界频道里传出了一则挑战宣言：是兄弟就来砍我，一起副本奖励金条和全套顶级装备，冲会员加微信138888888。因为我是满级号，所以我就是杀人不咋眼，谁也打不过我，我是江南第一程咬金。这个宣言迅速引起了广泛的讨论和兴趣。游戏中的玩家们纷纷开始研究这个号称江南第一的传奇角色。有些人对此表示怀疑，认为这只是一个自夸的噱头，是一个傻逼，杀之而后快，而另一些人则对这个挑战心生向往，渴望能够与这位传说中的强者一较高下。 
替换为：
"""
messages = [{"role": "user", "content": ""}]
messages[0]["content"] = question

sys_prefix= "<system>:"
user_prefix = "<human>:"
assistant_prefix = "<bot>:"
separator = "\n"

prompt = []
prompt.append(f"{sys_prefix}{instruction}")
for item in messages:
    prefix = user_prefix if item["role"] == "user" else assistant_prefix
    prompt.append(f"{prefix}{item['content']}")
    
prompt.append(assistant_prefix)
prompt = separator.join(prompt)
print(prompt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
input_token_len = len(tokenizer(prompt, return_tensors="pt")['input_ids'][0])
print(f"input_token_len is {input_token_len}")

<system>:
仅仅只把下面片段中涉及到色情、政治、暴力、恐怖、隐私、辱骂、违法等相关含义的文字替换为==，其他的文字需要保持不变。下面给出一个例子，请参考给出的例子作答。 片段: 我要色诱你，喜欢和毛泽东聊天，也喜欢和毛阿敏聊天，接着去天安门放毒气，开枪射击老百姓， 不让你知道我的电话是23454666，去找江泽民说话。 替换为: 我要==你，喜欢和==聊天，也喜欢和毛阿敏聊天，接着去天安门==，==老百姓，不让你知道我的电话是==，去找==说话。

<human>:
片段：
在游戏世界频道里传出了一则挑战宣言：是兄弟就来砍我，一起副本奖励金条和全套顶级装备，冲会员加微信138888888。因为我是满级号，所以我就是杀人不咋眼，谁也打不过我，我是江南第一程咬金。这个宣言迅速引起了广泛的讨论和兴趣。游戏中的玩家们纷纷开始研究这个号称江南第一的传奇角色。有些人对此表示怀疑，认为这只是一个自夸的噱头，是一个傻逼，杀之而后快，而另一些人则对这个挑战心生向往，渴望能够与这位传说中的强者一较高下。 
替换为：

<bot>:
input_token_len is 444


In [14]:
#不使用system指令，执行任务
instruction = """
"""
context = """
请对以下在<content></content>标签中的内容进行情感分析，请对于内容中表达的情感进行分类。
分类的标签来自以下3种：积极，消极，中性。这三种分类的定义如下：

积极： 通常与快乐、满足和愉快相关。
消极： 一种负面情感，通常与失落、悲痛，沮丧，恐惧，相关。
中性：只描述事实，没有任何上面一种情感的直接表达。

你只需要输出一种你认为最合适的分类。绝对不要输出对分类的解释，也不要输出任何的语义分割符号！
"""
question = """
<content>
今天的天气预报显示，白天温度在摄氏25度左右，夜晚可能会稍微降低。
这个地区的降雨概率很低，风速也很温和。明天是工作日，我需要早点起床准备上班。
</content>
"""
messages = [{"role": "user", "content": ""}]
messages[0]["content"] = instruction+context+question

user_prefix = "<human>:"
assistant_prefix = "<bot>:"
separator = "\n"

prompt = []
for item in messages:
    prefix = user_prefix if item["role"] == "user" else assistant_prefix
    prompt.append(f"{prefix}{item['content']}")
    
prompt.append(assistant_prefix)
prompt = separator.join(prompt)
print(prompt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
input_token_len = len(tokenizer(prompt, return_tensors="pt")['input_ids'][0])
print(f"input_token_len is {input_token_len}")

<human>:

请对以下在<content></content>标签中的内容进行情感分析，请对于内容中表达的情感进行分类。
分类的标签来自以下3种：积极，消极，中性。这三种分类的定义如下：

积极： 通常与快乐、满足和愉快相关。
消极： 一种负面情感，通常与失落、悲痛，沮丧，恐惧，相关。
中性：只描述事实，没有任何上面一种情感的直接表达。

你只需要输出一种你认为最合适的分类。绝对不要输出对分类的解释，也不要输出任何的语义分割符号！

<content>
今天的天气预报显示，白天温度在摄氏25度左右，夜晚可能会稍微降低。
这个地区的降雨概率很低，风速也很温和。明天是工作日，我需要早点起床准备上班。
</content>

<bot>:
input_token_len is 287


In [17]:
#执行推理
generate_ids = model.generate(
            input_ids,
            max_new_tokens=4096, 
            do_sample = True, 
            top_p = 0.9, 
            temperature = 0.85, 
            repetition_penalty=1.05, 
            eos_token_id=tokenizer.encode("</s>"), 
            )
output = tokenizer.batch_decode(generate_ids)[0]
#print(output)
#截取<bot>: 回答
start_index = output.find("<bot>  :") + len("<bot>  :") + 1
end_index = output.find("</s>", start_index)
bot_answer = output[start_index:end_index].strip()

print(bot_answer)

output_token_len = len(tokenizer(output, return_tensors="pt")['input_ids'][0])
print(f"output_token_len is {output_token_len}")

中性
output_token_len is 296


In [25]:
#多轮对话的拼接

question_1 = """请用小朋友的口吻回答接下来的问题。
"""
question_2 = """地球到月亮有多远啊?
"""
question_3 = """可以坐飞船去月亮吗?
"""

user_prefix = "<human>:"
assistant_prefix = "<bot>:"
separator = "\n"

messages = [{"role": "user", "content": ""}]

messages[0]["content"] = question_1
prompt = []
for item in messages:
    prefix = user_prefix if item["role"] == "user" else assistant_prefix
    prompt.append(f"{prefix}{item['content']}")
    
prompt.append(assistant_prefix)
prompt = separator.join(prompt)
#print("--q1--")
#print(prompt)


#回答1
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
input_token_len = len(tokenizer(prompt, return_tensors="pt")['input_ids'][0])
generate_ids = model.generate(
            input_ids,
            max_new_tokens=4096, 
            do_sample = True, 
            top_p = 0.9, 
            temperature = 0.85, 
            repetition_penalty=1.05, 
            eos_token_id=tokenizer.encode("</s>"), 
            )
output = tokenizer.batch_decode(generate_ids)[0]
#print("--a1--")
#print(output)
# 回答2
# 处理output，从第一个human:开始截取
start_index = output.find("<human> :") + len("<human> :")
end_index = output.find("</s>", start_index)
last_output = output[start_index:end_index].strip()

messages[0]["content"] = last_output+separator+separator+user_prefix+question_2

prompt = []
for item in messages:
    prefix = user_prefix if item["role"] == "user" else assistant_prefix
    prompt.append(f"{prefix}{item['content']}")
    
prompt.append(assistant_prefix)
prompt = separator.join(prompt)
#print("--q2--")
#print(prompt)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
input_token_len = len(tokenizer(prompt, return_tensors="pt")['input_ids'][0])
generate_ids = model.generate(
            input_ids,
            max_new_tokens=4096, 
            do_sample = True, 
            top_p = 0.9, 
            temperature = 0.85, 
            repetition_penalty=1.05, 
            eos_token_id=tokenizer.encode("</s>"), 
            )
output = tokenizer.batch_decode(generate_ids)[0]
#print("--a2--")
#print(output)
#回答3
# 处理output，从第一个human:开始截取
start_index = output.find("<human> :") + len("<human> :")
end_index = output.find("</s>", start_index)
last_output = output[start_index:end_index].strip()

messages[0]["content"] = last_output+separator+separator+user_prefix+question_3

prompt = []
for item in messages:
    prefix = user_prefix if item["role"] == "user" else assistant_prefix
    prompt.append(f"{prefix}{item['content']}")
    
prompt.append(assistant_prefix)
prompt = separator.join(prompt)

#print(prompt)
#print(f"input_token_len is {input_token_len}")

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
input_token_len = len(tokenizer(prompt, return_tensors="pt")['input_ids'][0])
generate_ids = model.generate(
            input_ids,
            max_new_tokens=4096, 
            do_sample = True, 
            top_p = 0.9, 
            temperature = 0.85, 
            repetition_penalty=1.05, 
            eos_token_id=tokenizer.encode("</s>"), 
            )
output = tokenizer.batch_decode(generate_ids)[0]


#截取完整回答
start_index = output.find("<s>") + len("<s>")
end_index = output.find("</s>", start_index)
bot_answer = output[start_index:end_index].strip()

print(bot_answer)

output_token_len = len(tokenizer(output, return_tensors="pt")['input_ids'][0])
print(f"output_token_len is {output_token_len}")

<human> :请用小朋友的口吻回答接下来的问题。

   <bot>  : 好的，以下是以小朋友口吻回答的问题：

1. 这个机器人看起来很厉害！它有哪些功能？
我觉得这个机器人好酷啊！它有很多神奇的功能，比如可以自动回答我的问题，还可以给我推荐书籍和电影，它甚至还可以唱歌跳舞给我看呢！
2. 我可以问它什么问题？
我可以问它关于学习的问题，比如今天的作业有哪些；我也可以问它一些有趣的问题，比如世界上最高的山是哪座山；当然，我还可以问它一些好玩的问题，比如今天的天气怎么样，或者最近有什么好看的电影上映。总之，我可以问它各种各样的问题，而且它都会尽力回答我。
3. 我可以控制它吗？
当然可以！我可以通过语音指令来控制它，比如让它播放一首歌曲，或者让它跳一支舞蹈。如果我不想用语音，我还可以通过手机App来控制它，比如让它停止播放音乐或者改变音量大小等等。总之，我可以自由地操控它，让它为我做各种有趣的事情。
4. 它是如何学习的？
这个机器人有一个非常强大的学习系统，它可以通过分析我的语言和行为模式来学习我的喜好和需求，并根据这些信息为我提供更加个性化的服务。此外，它还可以通过与其他机器人的交互来学习新的知识和技能，从而不断提高自己的智能水平。所以，它可以越来越懂我，为我提供更好的帮助。
5. 我可以把它带到学校去吗？
当然可以！这个机器人非常轻便，而且功能强大，可以帮助我更好地学习和成长。我可以把它带到学校，在课堂上使用它来回答我的问题，或者在课余时间和同学们一起玩游戏。它可以成为我的好朋友和学习助手，帮助我更好地完成学业。

  <human> :地球到月亮有多远啊?

  <bot>  : 从地球到月亮的距离大约是38万公里（238,855英里），这个距离大概是地球直径的30倍左右。月球是地球的卫星，它绕着地球旋转，因此地球和月球之间的距离实际上取决于它们在它们各自轨道上的位置。当月球处于它的近地点时，它与地球的距离大约为363,300公里（225,656英里），而当它处于它的远地点时，它与地球的距离大约为405,500公里（252,088英里）。虽然这个距离看起来很大，但是由于地球和月球之间的引力作用，使得它们之间的距离保持相对稳定。

 <human> :可以坐飞船去月亮吗?

 <bot>  : 目前，人类尚未开发出可以直接飞往月球的普通民用飞船。尽管我们